# Parallel Python & Dask

## James Percival

### 29th October 2020 - Version 1.0.1

- Final lecture of ACSE 1
- Not directly assessed here
- Will be useful in rest of course
- Particularly useful in group projects
- Outside reading recommended.

```
conda env create -f environment.yml
conda activate dask-tutorial
jupyter notebook lecture.ipynb
```

## What is Parallel computation?

> In the simplest sense, parallel computing is the simultaneous use of multiple compute resources to solve a computational problem:
>
>    A problem is broken into discrete parts that can be solved concurrently
    Each part is further broken down to a series of instructions
    Instructions from each part execute simultaneously on different processors
    An overall control/coordination mechanism is employed 
    
[_Blaise Barney, Lawrence Livermore National Laboratory_](https://computing.llnl.gov/tutorials/parallel_comp/#Whatis)

Two ways of playing polyphonic musics

- one performer plays many notes - eg. pianist
- many performers play one note each - eg. orchestra

- One instance of program executing many sections - threads
- Many instances of programing executed once each - procesess

## Threads

- Cheap to create/destroy
- Share data through program memory
- Needs locks to control shared resources
- Limited to one computer
- What your web browser does a lot of

## Processes

- Higher overhead to fork process
- Communicate through network or external process
- Often have own copies of resources
- Can be distributed across multiple machines.
- What many HPC programs do a lot of.

## Writing parallel algorithms

- Not all problems parallelise (easily).
- Simplest ones can be written as
  ```
  for every X do y
  ```
- Worst have each step depend nonlinearly on the last (e.g. hash caching)
- Will discuss more in ACSE 6

### syncronous versus asyncronous

- When interacting with something external code can:
  - Sit and wait (syncronous)
  - Come back later (asyncronous)
  - `async` and `await` keywords in Python

In [ ]:
import time
import asyncio

def fun1(n, m):
    a = [_**2 for _ in range(m)]
    time.sleep(n)
    return a

In [ ]:
t1 = time.time()
a = fun1(2.0, 1000000)
print(time.time()-t1)

In [ ]:
async def _fun2a(n):
    t = await asyncio.sleep(n)
    return t

async def _fun2b(m):
    a = [_**2 for _ in range(m)]
    return a

async def fun2(n, m):
    a = await asyncio.gather(_fun2a(n), _fun2b(m))
    return a[0]

In [ ]:
t1 = time.time()
a = await fun2(2.0, 1000000)
print(time.time()-t1)

Asyncronous routines are:
- Not usually default
- Hard to get right
- Frequently surprising 
- Hard to debug
- can sometimes massively decrease runtime

### Lazy evalution

- Related to asyncronous computation
- Fastest code is code that does not run
- Avoid calculating unneeded intermediate values
- Opposite standpoint is called "eager" evaluation (Python default)

In [ ]:
from functools import total_ordering

@total_ordering
class Lazy_factorial:
    """Example of a factorial class with lazy evaluation & caching."""
    
    def __init__(self, n):
        """Factorial class, returns n factorial when evaluated."""
        
        self._result = None
        self.n = n
        
    def compute(self):
        """Final evaluation, with cache."""
        if not self._result:
            self._result = self.factorial(self.n)
        return self._result
    
    def factorial(self, n):
        """Compute factorial with loop."""
        res = 1
        for _ in range(1, n+1):
            res *= _
        return res
    
    def __repr__(self):
        return f'Lazy_factorial({self.n})'
    
    def __lt__(self, other):
        return self.n < other.n
    
    def __eq__(self, other):
        return self.n == other.n
    

In [ ]:
%time a=Lazy_factorial(1)
%time b=Lazy_factorial(1000)

In [ ]:
%time a.compute()
%time b.compute()
%time b.compute()

In [ ]:
%time Lazy_factorial(100)<Lazy_Factorial(300)

## `threading` module

Provides basic, builtin access to uses multiple threads.

In [ ]:
import threading
import multiprocessing
import numpy as np

def do_nothing(n):
    print(f"Run {n}.")
    time.sleep(1)

Now to actually use it.

In [ ]:
%%time
for i in range(10):
    do_nothing(i)

In [ ]:
%%time
threads = []
for i in range(10):
    threads.append(threading.Thread(target=do_nothing, args=(i,)))
    threads[-1].start()
for thread in threads:
    thread.join()

- IO is messed up.
- Threads don't wait for each other
- Can fix with a lock

In [ ]:
def do_nothing_with_lock(n, lock):
    lock.acquire()
    print(f"Run {n}.")
    lock.release()
    time.sleep(1)

In [ ]:
%%time
threads = []
lock = threading.Lock()
for i in range(10):
    threads.append(threading.Thread(target=do_nothing_with_lock, args=(i, lock)))
    threads[-1].start()
for thread in threads:
    thread.join()

Sleep is boring. Lets try `numpy` and regular Python.

In [ ]:
def do_something_in_numpy(arr):
    arr[:] = np.sin(arr)
    
def do_something_in_python(n):
    print(f"Run {n}.")
    a = [_**2 for _ in range(1000000)]

#### Serial version

In [ ]:
arr0 = np.arange(1000000, dtype=float)
arr1 = arr0.copy()
%time do_something_in_numpy(arr1)

In [ ]:
arr2 = arr0.copy()

In [ ]:
%%time
threads = []
n = 2
N = arr2.size//n
for i in range(n):
    threads.append(threading.Thread(target=do_something_in_numpy,
                                    args=(arr2[i*N:(i+1)*N],)))
    threads[-1].start()
for thread in threads:
    thread.join()

In [ ]:
(arr1 == arr2).all()

Try Python version in serial

In [ ]:
%%time
for i in range(10):
    do_something_in_python(i)

In [ ]:
%%time
threads = []
for i in range(10):
    threads.append(threading.Thread(target=do_something_in_python, args=(i,)))
    threads[-1].start()
for thread in threads:
    thread.join()

Example of the GIL (Global Interpreter Lock). 

For thread safety & to maximise serial speed, Python code usually runs one step in one thread at a time.

So except for specific problems (eg. numpy, pandas, IO, GUIs) threads aren't the answer

Enter `multiprocessing`.

In [ ]:
%%time
processes = []
for i in range(10):
    processes.append(multiprocessing.Process(target=do_something_in_python, args=(i,)))
    processes[-1].start()
for process in processes:
    process.join()

Comes with a few options to make life a bit simpler with common patterns.

In [ ]:
def fn(x):
    return x**3

a = range(100)

pool = multiprocessing.Pool(processes=4)
pool.map(fn, a)

# Enter Dask

- Dynamic task scheduler
- Reduce boilerplate on single machine
- Simplify running across multiple machines
- Good links to Pandas

How to start scheduler (lots of keyword options available)

In [ ]:
from dask.distributed import Client

## n_workers could be the number of threads or processes
client = Client(n_workers=4, processes=True)

Now let's switch over to the Dask tutorial text.